In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import time
import pandas as pd

In [ ]:
def generar_capas(n_capas: int = 128, 
                  r: float = 3.442619855899, 
                  v: float = 9.91256303526217e-3) -> np.ndarray:
    """
    Calcula los bordes x_i de las capas del ziggurat para la normal estándar (lado positivo).
    
    Implementación basada en Marsaglia & Tsang (2000), sección 3.
    
    Args:
        n_capas: Número de capas (típicamente 64, 128, o 256)
        r: Borde derecho (x_{N-1}), del paper
        v: Área común de cada rectángulo, del paper
    
    Returns:
        Array de forma (n_capas,) con los bordes:
        - x_list[0] = 0 (borde izquierdo)
        - x_list[i] = borde derecho de la capa i
        - x_list[-1] = r (borde de la cola)
    """
    # Función densidad sin normalizar, es decir, sin 1/sqrt(2*pi) (lado positivo de N(0,1))
    f = lambda x: np.exp(-0.5 * x * x)
    
    # Inversa de f: f^{-1}(y) = sqrt(-2 * ln(y))
    f_inv = lambda y: np.sqrt(-2.0 * np.log(y))
    
    # Array de bordes (x_0, x_1, ..., x_{N-1})
    x_list = np.zeros(n_capas, dtype=np.float64)
    x_list[-1] = r  # x_{N-1} = r (borde de la cola)
    
    # Iteración hacia atrás: x_i = f^{-1}(v/x_{i+1} + f(x_{i+1}))
    # Basado en la relación: x_i * [f(x_{i-1}) - f(x_i)] = v
    for i in range(n_capas - 2, 0, -1):
        y = f(x_list[i + 1]) + v / x_list[i + 1]
        x_list[i] = f_inv(y)
    
    # x_0 es el borde izquierdo (siempre 0)
    x_list[0] = 0.0
    
    return x_list

In [ ]:
def generar_tablas_precalculadas(x_list: np.ndarray, v: float):
    """
    Construye las tablas k, w, f del Ziggurat para optimizar la generación.
    
    Estas tablas permiten que el test rápido (paso 2 del algoritmo) se ejecute
    con solo 2 accesos a memoria y 1 comparación de enteros.
    
    Tablas generadas:
    - k[i]: Umbral entero para el test rápido (uint32)
    - w[i]: Factor de escala para convertir entero → float (float64)
    - f[i]: Valores precalculados de la densidad f(x_i) (float64)
    
    Args:
        x_list: Array con los bordes x_i de las capas (longitud N)
        v: Área común de cada rectángulo
    
    Returns:
        (k_list, w_list, f_list): Tupla con las tres tablas
    
    Notas sobre el diseño:
    - M = 2^31 porque trabajamos con enteros int32 (rango: -2^31 a 2^31-1)
    - La capa i=0 es especial: combina rectángulo base + cola infinita
    """
    x_list = np.asarray(x_list, dtype=np.float64)
    N = x_list.size
    
    if N < 2:
        raise ValueError("Se requieren al menos 2 bordes: x_0≈0 y x_{N-1}=r.")

    # Tablas a rellenar
    f_list = np.exp(-0.5 * x_list * x_list)
    k_list = np.zeros(N, dtype=np.uint32)
    w_list = np.zeros(N, dtype=np.float64)

    # Escala entera M = 2^31 
    M = float(2**31)


    # (A) Filas i>=1: relaciones directas entre bordes adyacentes
    w_list[1:] = x_list[1:] / M
    with np.errstate(divide="ignore", invalid="ignore"):
        # Calcular ratio x_{i-1} / x_i para todas las capas
        ratio = x_list[:-1] / x_list[1:]
        # Manejar casos donde x_i podría ser 0 (solo x_0 en realidad)
        ratio = np.where(x_list[1:] > 0, ratio, 0.0)
    # Escalar a enteros y truncar: k[i] = floor(M * ratio)
    k_list[1:] = np.floor(M * ratio).astype(np.uint32)

    # (B) Fila i=0 (base + cola): usa q = v / f(r)
    r = x_list[-1]
    fr = f_list[-1]

    if fr <= 0.0:
        raise ValueError("f(r) es 0")
    
    q = v / fr
    k_list[0] = np.uint32(np.floor((r / q) * M))
    w_list[0] = q / M

    return k_list, w_list, f_list

In [ ]:
def cola_normal(r: float, rng: np.random.Generator, u1_inicial: float = None) -> float:
    """
    Genera un valor de la cola derecha de N(0,1) (x > r).
    Usa el método de Marsaglia (1963) optimizado.
    
    Este método transforma la cola exponencial en el intervalo unitario
    y aplica rechazo para generar valores > r de la normal.
    
    Args:
        r: límite de la cola (típicamente r ≈ 3.44 para N=128)
        rng: generador de números aleatorios
        u1_inicial: primer uniforme
    
    Returns:
        float: valor x > r siguiendo la cola de N(0,1)
    """

    # Primera iteración: reutilizar u1_inicial si está disponible
    # Esto ahorra una llamada al RNG cuando venimos del test rápido fallido
    u1 = u1_inicial if u1_inicial is not None else rng.random()
    
    while True:
        # Generar segundo uniforme
        u2 = rng.random()

        # Transformación de Marsaglia (1963)
        x = -np.log(u1) / r
        y = -np.log(u2)
        
        # Test de aceptación de Marsaglia:
        # Aceptar si 2*y > x², equivalente a y+y > x*x
        # Esto acepta puntos bajo la curva exp(-x²/2) en la cola
        # Eficiencia: ~88% de aceptación para r=3.44
        if (y + y) > (x * x):
            return r + x
        
        # Rechazado: genera nuevo u1 para siguiente iteración
        u1 = rng.random()

In [ ]:
def ziggurat_norm_one(
    rng: np.random.Generator,
    k: np.ndarray,        # uint32, shape (N,)
    w: np.ndarray,        # float64, shape (N,)
    f: np.ndarray,        # float64, shape (N,), f[i]=exp(-xi[i]^2/2)
    xi: np.ndarray,       # float64, shape (N,), bordes lado positivo
    r: float              # límite derecho (xi[-1])
) -> float:
    """
    Genera UNA muestra ~ N(0,1) usando el método Ziggurat (Marsaglia & Tsang, 2000),
    con tablas precalculadas (k, w, f, xi). Devuelve un float.

    Reglas:
      - Test rápido: |hz| < k[i]  -> x = hz * w[i] (aceptado)
      - Si i == 0 y falla: cola (x>r) con Marsaglia 1963; aplicar signo de hz
      - Si i > 0 y falla: tiras (rechazo local) con alturas tabuladas f[i-1], f[i]
    """
    
    while True:
        # 1) Genera UN solo entero de 32 bits con signo
        hz = rng.integers(np.iinfo(np.int32).min, np.iinfo(np.int32).max, 
                         dtype=np.int32, endpoint=True)
        
        # 2) Índice de capa (usa los últimos 7 bits para N=128)
        iz = abs(hz) & 127
        
        # 3) Test rápido: |hz| < k[iz] ?
        abs_hz = np.uint32(abs(int(hz)))
        if abs_hz < k[iz]:
            # Acepta inmediatamente
            return float(hz) * float(w[iz])
        
        # 4) Test falló - determinar signo una sola vez
        sign = 1.0 if hz >= 0 else -1.0
        
        # 5) Casos especiales según el índice
        if iz == 0:
            # COLA: genera valor > r usando el método de Marsaglia
            u1_inicial = abs_hz / (2.0**32)
            x_tail = cola_normal(r, rng, u1_inicial)
            return sign * x_tail
        
        else:
            # TIRAS (strips): usa hz para formar x
            # x = hz * w[iz] ya está calculado implícitamente
            x = float(hz) * float(w[iz])
            
            # Genera UN solo uniforme para el test de rechazo
            u = rng.random()
            
            # Test: f[iz-1] - f[iz] * U < f(x) - f[iz]
            # Equivalente a: [f(x_{i-1}) - f(x_i)] * U < f(x) - f(x_i)
            # (ver paso 4 del algoritmo en la página 3 del paper)
            lhs = (float(f[iz - 1]) - float(f[iz])) * u
            rhs = np.exp(-0.5 * x * x) - float(f[iz])
            
            if lhs < rhs:
                return x
            
            # Si rechaza, el while True reinicia con nuevo hz


In [ ]:
def generar_muestras_ziggurat(n_muestras: int, seed: int = 42) -> np.ndarray:
    """
    Genera n_muestras usando el método Ziggurat.
    
    Args:
        n_muestras: Cantidad de números aleatorios a generar
        seed: Semilla para reproducibilidad
    
    Returns:
        Array de forma (n_muestras) con valores ~ N(0,1)
    """
    # 1. Inicializar generador
    mt = np.random.MT19937(seed=seed)
    rng = np.random.Generator(mt)
    
    # 2. Generar capas y tablas (solo una vez)
    print("Inicializando Ziggurat...")
    r = 3.442619855899
    v = 9.91256303526217e-3
    x_list = generar_capas(n_capas=128, r=r, v=v)
    k, w, f = generar_tablas_precalculadas(x_list, v)
    print(f"Tablas generadas para {len(x_list)} capas")
    
    # 3. Generar muestras
    print(f"Generando {n_muestras:,} muestras...")
    start_time = time.time()
    
    muestras = np.array([ziggurat_norm_one(rng, k, w, f, x_list, r) 
                        for _ in range(n_muestras)])
    
    elapsed = time.time() - start_time
    print(f"Generadas en {elapsed:.3f} segundos ({n_muestras/elapsed:,.0f} muestras/seg)")
    
    return muestras